In [ ]:
import os
os.chdir("..")

In [ ]:
from src.data.data_process import DataReg
import arviz as az
import bambi as bmb
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import os

load_dotenv()

az.style.use("arviz-darkgrid")


In [ ]:
# Separate lists for each group
local_results = []
foreign_results = []

# First pass: load and rename data
for file in os.listdir("data/processed/processed"):
    results = az.from_netcdf(f"data/processed/processed/{file}")
    parts = file[:-3].split("_")
    
    name = f"Kaits for Naics {parts[2]}"
    name_dict = {'log_k_index': name}
    results.rename(name_dict, inplace=True)

    if parts[1] == "local":
        local_results.append((results, name))
    else:
        foreign_results.append((results, name))

# Figure size based on number of plots
n_local = len(local_results)
n_foreign = len(foreign_results)
n_rows = max(n_local, n_foreign)

fig, axes = plt.subplots(n_rows, 2, figsize=(12, 3 * n_rows), squeeze=False)

# Plot local (column 0)
for i, (idata, varname) in enumerate(local_results):
    az.plot_posterior(idata, var_names=[varname], ax=axes[i, 0])
    axes[i, 0].set_title(f"Local: {varname}")

# Plot foreign (column 1)
for i, (idata, varname) in enumerate(foreign_results):
    az.plot_posterior(idata, var_names=[varname], ax=axes[i, 1])
    axes[i, 1].set_title(f"Foreign: {varname}")

# Hide unused axes
for i in range(n_rows):
    if i >= n_local:
        axes[i, 0].axis('off')
    if i >= n_foreign:
        axes[i, 1].axis('off')

# Overall formatting
fig.suptitle("Posterior Distributions of log_k_index by Group", fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()

In [ ]:
results = az.from_netcdf("data/processed/processed/results_local_31-33.nc")
az.plot_trace(results)

In [ ]:
az.summary(results)

In [ ]:


fig, axes = plt.subplots(1, 2, figsize=(12, 4))

az.plot_posterior(results, var_names=["log_k_index"], ax=axes[0])
az.plot_posterior(results2, var_names=["log_k_index"], ax=axes[1])

axes[0].set_title("Empresas PR")
axes[1].set_title("Empresas Foreaneas")

plt.tight_layout()
plt.show()